In [1]:
import csv
import json
import random
import re
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns

import torch

from datasets import (
    Dataset,
    DatasetDict,
    concatenate_datasets,
    load_dataset,
    load_from_disk,
)
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
    set_seed,
)

import evaluate

from nameparser import HumanName
from names_dataset import NameDataset, NameWrapper
from ethnicseer import EthnicClassifier
import nltk
from nltk.corpus import wordnet as wn

import pycountry_convert as pc
import pycountry
import pickle

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, cohen_kappa_score

from transformers import BertTokenizerFast, BertForTokenClassification
from datasets import ClassLabel
from evaluate import load as load_metric
from sklearn.metrics import cohen_kappa_score
from itertools import combinations
import krippendorff

In [2]:
model_name = "bert-base-cased"

In [5]:
conll_main = load_from_disk("../splits/conll_main")
conll_clean = load_from_disk("../splits/conll_clean")

ontonotes_main = load_from_disk("../splits/ontonotes_main")
ontonotes_clean = load_from_disk("../splits/ontonotes_clean")

# Load GPU

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Tokenisation & Alignment

In [7]:
ontonotes_id_to_label = {
    0: "O", 1: "B-CARDINAL", 2: "B-DATE", 3: "I-DATE", 4: "B-PERSON", 5: "I-PERSON",
    6: "B-NORP", 7: "B-GPE", 8: "I-GPE", 9: "B-LAW", 10: "I-LAW", 11: "B-ORG", 12: "I-ORG",
    13: "B-PERCENT", 14: "I-PERCENT", 15: "B-ORDINAL", 16: "B-MONEY", 17: "I-MONEY",
    18: "B-WORK_OF_ART", 19: "I-WORK_OF_ART", 20: "B-FAC", 21: "B-TIME", 22: "I-CARDINAL",
    23: "B-LOC", 24: "B-QUANTITY", 25: "I-QUANTITY", 26: "I-NORP", 27: "I-LOC",
    28: "B-PRODUCT", 29: "I-TIME", 30: "B-EVENT", 31: "I-EVENT", 32: "I-FAC",
    33: "B-LANGUAGE", 34: "I-PRODUCT", 35: "I-ORDINAL", 36: "I-LANGUAGE"
}

conll_label_to_id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3,
                     'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in conll_label_to_id.items()}

ontonotes_to_conll_entity = {
    "PERSON": "PER", "ORG": "ORG", "GPE": "LOC", "LOC": "LOC",
    "NORP": "MISC", "FAC": "MISC", "EVENT": "MISC", "WORK_OF_ART": "MISC",
    "LAW": "MISC", "PRODUCT": "MISC", "LANGUAGE": "MISC",
    "DATE": None, "TIME": None, "PERCENT": None, "MONEY": None,
    "QUANTITY": None, "ORDINAL": None, "CARDINAL": None
}

In [8]:
def process_data(data_list):

    def process_single(data):
        word_ids = data['word_ids']
        predictions = data['predictions']
        gold = data['gold']
        tokenized_tokens = data['tokens']

        word_ids = [a for a in word_ids if a is not None]

        processed_predictions = []
        processed_gold = []

        current_word_id = None
        current_predictions = []
        current_gold = []

        for idx, word_id in enumerate(word_ids):
            if word_id != current_word_id:
                if current_predictions:
                    processed_predictions.append(
                        Counter(current_predictions).most_common(1)[0][0])
                    processed_gold.append(
                        Counter(current_gold).most_common(1)[0][0])

                current_word_id = word_id
                current_predictions = [predictions[idx]]
                current_gold = [gold[idx]]
            else:
                current_predictions.append(predictions[idx])
                current_gold.append(gold[idx])

        if current_predictions:
            processed_predictions.append(
                Counter(current_predictions).most_common(1)[0][0])
            processed_gold.append(
                Counter(current_gold).most_common(1)[0][0])

        return processed_predictions, processed_gold

    processed_predictions_list = []
    processed_gold_list = []

    for data in data_list:
        processed_predictions, processed_gold = process_single(data)
        processed_predictions_list.append(processed_predictions)
        processed_gold_list.append(processed_gold)

    return processed_predictions_list, processed_gold_list


def evaluate_predictions(p, test_data):
    predictions, labels, _ = p

    pred_indices = [np.argmax(p, axis=-1) for p in predictions]
    label_indices = labels

    pred_tags = [[id2label[p] for p, l in zip(p_seq, l_seq) if l != -100]
                 for p_seq, l_seq in zip(pred_indices, label_indices)]
    gold_tags = [[id2label[l] for l in l_seq if l != -100]
                 for l_seq in label_indices]

    def add_preds(example, idx):
        length = len(example['word_ids'])
        example['predictions'] = pred_tags[idx][:length]
        example['gold'] = gold_tags[idx][:length]
        return example

    test_data = test_data.map(add_preds, with_indices=True)

    length = len(test_data['predictions'][0])

    pred, gold = process_data(test_data)

    flat_pred = [label for seq in pred for label in seq]
    flat_gold = [label for seq in gold for label in seq]

    print(classification_report(flat_gold, flat_pred, zero_division=0))

    return (flat_pred, flat_gold)

# Model

In [9]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG',
              'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        return_special_tokens_mask=True,
        return_offsets_mapping=True,
    )
    all_word_ids = []
    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        all_word_ids.append(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_idx])
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    tokenized_inputs["word_ids"] = all_word_ids
    return tokenized_inputs

In [10]:
conll_main = conll_main.map(tokenize_and_align_labels, batched=True)
conll_clean = conll_clean.map(tokenize_and_align_labels, batched=True)
ontonotes_main = ontonotes_main.map(tokenize_and_align_labels, batched=True)
ontonotes_clean = ontonotes_clean.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4149 [00:00<?, ? examples/s]

Map:   0%|          | 0/15343 [00:00<?, ? examples/s]

# Deciding params

In [11]:
train_data = conll_main
test_data = conll_clean

In [12]:
train_data_name = 'conll'

In [13]:
mod = BertForTokenClassification.from_pretrained(
    model_name, num_labels=len(label_list))

metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)


data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=f"./output/{model_name}",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True,
    save_strategy="no",
)

trainer = Trainer(
    model=mod,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


C:\Users\NIKHIL\AppData\Local\Temp\ipykernel_23684\1248433719.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Step,Training Loss



KeyboardInterrupt



In [ ]:
trainer.save_model(f"./saved_model/{model_name}_{train_data_name}_self")

In [ ]:
predictions = trainer.predict(test_data)

In [ ]:
with open(f'./results/{model_name}_{train_data_name}_results.pkl', 'wb') as f:
    pickle.dump(predictions, f)

In [ ]:
x, y = evaluate_predictions(predictions, test_data)


Map:   6%|▌         | 3423/61371 [00:00<00:14, 3949.93 examples/s]


Map:   7%|▋         | 4000/61371 [00:00<00:15, 3806.36 examples/s]


Map:   8%|▊         | 4905/61371 [00:01<00:11, 4906.18 examples/s]


Map:   9%|▉         | 5741/61371 [00:01<00:12, 4630.69 examples/s]


Map:  10%|█         | 6380/61371 [00:01<00:12, 4241.78 examples/s]

...


Map:  98%|█████████▊| 60000/61371 [00:16<00:00, 3023.13 examples/s]


Map:  99%|█████████▉| 60705/61371 [00:17<00:00, 3675.60 examples/s]


Map: 100%|██████████| 61371/61371 [00:19<00:00, 909.80 examples/s] 


Map: 100%|██████████| 61371/61371 [00:19<00:00, 3162.99 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.83      0.89      0.86     17495
      B-MISC       0.64      0.73      0.68     10657
       B-ORG       0.63      0.68      0.65     13041
       B-PER       0.90      0.93      0.91     15547
       I-LOC       0.71      0.66      0.68      5367
      I-MISC       0.66      0.37      0.47      7305
       I-ORG       0.90      0.71      0.79     18313
       I-PER       0.95      0.88      0.91     11086
           O       0.99      0.99      0.99   1011383

    accuracy                           0.97   1110194
   macro avg       0.80      0.76      0.77   1110194
weighted avg       0.97      0.97      0.97   1110194



## Stanford

In [ ]:
df = pd.read_excel(
    "./Guided-Adversarial-Augmentation-main/Guided-Adversarial-Augmentation-main/data/data/conll2003/challenge_set.xlsx", header=None)

In [ ]:
examples = []

i = 0
while i < len(df):
    row = df.iloc[i]
    if str(row[0]).startswith("GUID"):

        guid = str(df.iloc[i][1]).strip()

        try:
            quality = int(str(df.iloc[i+1][1]).strip())
        except (ValueError, TypeError):
            quality = 999

        try:
            aug_type = int(str(df.iloc[i+2][1]).strip())
        except (ValueError, TypeError):
            aug_type = 999

        tokens_row = df.iloc[i+3].dropna().tolist()[1:]
        labels_row = df.iloc[i+4].dropna().tolist()[1:]
        labels_row = [label.strip()
                      for label in labels_row if label.strip() != ""]

        if len(tokens_row) == len(labels_row):
            examples.append({
                "guid": guid,
                "quality": quality,
                "aug_type": aug_type,
                "tokens": tokens_row,
                "labels": labels_row
            })

        i += 6
    else:
        i += 1

challenge_dataset = Dataset.from_list(examples)

In [ ]:
conll_label_to_id = {
    'O': 0,
    'B-PER': 1, 'I-PER': 2,
    'B-ORG': 3, 'I-ORG': 4,
    'B-LOC': 5, 'I-LOC': 6,
    'B-MISC': 7, 'I-MISC': 8,
}


def encode_labels(example):
    example["labels"] = [conll_label_to_id.get(
        label, 0) for label in example["labels"]]
    return example

In [ ]:
challenge_dataset = challenge_dataset.map(encode_labels)


Map: 100%|██████████| 1418/1418 [00:00<00:00, 10944.56 examples/s]

In [21]:
stanford_encoded = challenge_dataset.map(
    tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/1418 [00:00<?, ? examples/s]


Map:  71%|███████   | 1000/1418 [00:00<00:00, 2132.63 examples/s]


Map: 100%|██████████| 1418/1418 [00:00<00:00, 2498.01 examples/s]


Map: 100%|██████████| 1418/1418 [00:00<00:00, 2388.34 examples/s]

In [22]:
stanford_results = trainer.predict(stanford_encoded)

In [23]:
with open(f'./results/{model_name}_{train_data_name}_stanford', 'wb') as f:
    pickle.dump(stanford_results, f)

In [24]:
x, y = evaluate_predictions(stanford_results, stanford_encoded)


Map:   0%|          | 0/1418 [00:00<?, ? examples/s]


Map:  46%|████▌     | 646/1418 [00:00<00:00, 6385.92 examples/s]


Map: 100%|█████████▉| 1414/1418 [00:00<00:00, 5309.18 examples/s]


Map: 100%|██████████| 1418/1418 [00:00<00:00, 4913.02 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.63      0.86      0.72       539
      B-MISC       0.93      0.92      0.93       319
       B-ORG       0.70      0.85      0.77      1214
       B-PER       0.88      0.64      0.74      1067
       I-LOC       0.38      0.86      0.53        79
      I-MISC       0.87      0.88      0.87        82
       I-ORG       0.87      0.94      0.90      2026
       I-PER       1.00      0.63      0.77       939
           O       1.00      0.99      1.00     22466

    accuracy                           0.96     28731
   macro avg       0.81      0.84      0.80     28731
weighted avg       0.96      0.96      0.96     28731



## Personal

In [25]:
nd = NameDataset()
country_codes = nd.get_country_codes()

continent_names = {
    'AF': 'Africa',
    'NA': 'North America',
    'OC': 'Oceania',
    'AN': 'Antarctica',
    'AS': 'Asia',
    'EU': 'Europe',
    'SA': 'South America',
}

continent_to_code = defaultdict(list)

continent_to_countries = defaultdict(list)

for code in country_codes:
    try:
        alpha_2 = code.alpha_2
        country = pycountry.countries.get(alpha_2=alpha_2)
        if not country:
            continue

        continent_code = pc.country_alpha2_to_continent_code(alpha_2)
        continent = continent_names[continent_code]

        continent_to_code[continent].append(alpha_2)
        continent_to_countries[continent].append((alpha_2, country.name))

    except Exception as e:
        print(f"Skipping {code} due to error: {e}")
        continue

In [26]:
def get_names(country, n=10):
    names_dict = nd.get_top_names(n=n, country_alpha2=country)
    names = []
    if country in names_dict:
        country_names = names_dict[country]
        for gender in ['M', 'F']:
            if gender in country_names:
                names.extend(country_names[gender])
    return list(set(names))


def tokenize(text):
    tokens = re.findall(r"\w+|[^\w\s]", text, re.UNICODE)
    return tokens


def tag_tokens(tokens, person_names):
    tags = ["O"] * len(tokens)
    for name in person_names:
        name_tokens = name.split()
        n_len = len(name_tokens)
        for i in range(len(tokens) - n_len + 1):
            if tokens[i:i + n_len] == name_tokens:
                tags[i] = "B-PER"
                for j in range(i + 1, i + n_len):
                    tags[j] = "I-PER"
    return tags


def generate_challenge_dataset(num_samples=300):
    dataset = []
    failures = 0
    max_failures = 1000

    while len(dataset) < num_samples:
        if failures >= max_failures:
            print(f"Stopped after {failures} failed attempts.")
            break

        country = random.choice(chosen_countries)
        names = get_names(country, n=10)

        # Filter names: keep only those also in common_nouns
        filtered_names = [
            name for name in names if name.lower() in common_nouns]

        if not filtered_names:
            failures += 1
            if failures % 10 == 0:
                print(f"{failures} failed attempts so far.")
            continue

        # Pick one or two names as needed
        if len(filtered_names) == 1:
            name1 = filtered_names[0]
            name2 = None
        else:
            name1, name2 = random.sample(filtered_names, 2)

        template = random.choice(sentence_templates)

        if "{name2}" in template and name2 is None:
            sentence = template.format(name=name1, name2=name1)
            person_names = [name1]
        elif "{name2}" in template:
            sentence = template.format(name=name1, name2=name2)
            person_names = [name1, name2]
        else:
            sentence = template.format(name=name1)
            person_names = [name1]

        tokens = tokenize(sentence)
        tags = tag_tokens(tokens, person_names)
        dataset.append(list(zip(tokens, tags)))
        print(len(dataset))

    return dataset


def save_to_csv(dataset, filepath="default_name.csv"):
    with open(filepath, mode="w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["sentence_id", "token", "tag"])
        for idx, sentence in enumerate(dataset):
            for token, tag in sentence:
                writer.writerow([idx, token, tag])

In [27]:
def evaluate_csv(file, common=False):
    print('evaluating', file)
    scripted_challenge = pd.read_csv(file)

    examples = []
    for sentence_id, group in scripted_challenge.groupby("sentence_id"):
        tokens = group["token"].tolist()
        tags = group["tag"].tolist()
        examples.append({"tokens": tokens, "ner_tags": tags})

    unique_tags = sorted(set(tag for ex in examples for tag in ex["ner_tags"]))

    for ex in examples:
        ex["labels"] = [conll_label_to_id[tag] for tag in ex["ner_tags"]]

    challenge_dataset = Dataset.from_list(examples)
    tokenized = challenge_dataset.map(tokenize_and_align_labels, batched=True)
    a = trainer.predict(tokenized)

    if common:
        with open(f'./results/{model_name}_{train_data_name}_challenge_common.pkl', 'wb') as f:
            pickle.dump(a, f)
    else:
        with open(f'./results/{model_name}_{train_data_name}_challenge_{file[:-3]}.pkl', 'wb') as f:
            pickle.dump(a, f)

    x, y = evaluate_predictions(a, tokenized)
    print()
    print('-'*30)

In [28]:
for x in list(continent_to_code.keys()):
    evaluate_csv(x+'_ner_challenge.csv')

evaluating North America_ner_challenge.csv



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 7929.91 examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 7634.21 examples/s]

/scistor/guest/lre040/miniconda3/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/scistor/guest/lre040/miniconda3/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 8673.34 examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 8431.34 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         0
       B-ORG       0.00      0.00      0.00         0
       B-PER       1.00      0.91      0.95      1148
       I-LOC       0.00      0.00      0.00         0
           O       1.00      0.98      0.99      9548

    accuracy                           0.97     10696
   macro avg       0.40      0.38      0.39     10696
weighted avg       1.00      0.97      0.99     10696


------------------------------


In [29]:
evaluate_csv('./common_nouns_challenge.csv', True)

evaluating ./common_nouns_challenge.csv



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 7812.19 examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 7495.83 examples/s]

/scistor/guest/lre040/miniconda3/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 8682.39 examples/s]


Map: 100%|██████████| 1000/1000 [00:00<00:00, 8448.88 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         0
       B-ORG       0.00      0.00      0.00         0
       B-PER       1.00      0.94      0.97      1168
           O       1.00      1.00      1.00      9336

    accuracy                           0.99     10504
   macro avg       0.50      0.49      0.49     10504
weighted avg       1.00      0.99      1.00     10504


------------------------------


In [30]:
seeds = [33, 42, 57, 106, 812, ]
results = []

for seed in seeds:
    set_seed(seed)

    seed_model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=len(conll_label_to_id)
    )

    seed_args = TrainingArguments(
        output_dir=f"./output/{seed}",
        seed=seed,
        eval_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        report_to="none",
        fp16=True,
        logging_steps=1000,
        save_strategy="no",
    )

    seed_trainer = Trainer(
        model=seed_model,
        args=seed_args,
        train_dataset=train_data,
        eval_dataset=test_data,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    seed_trainer.train()

    pred = seed_trainer.predict(test_data)

    x, y = evaluate_predictions(pred, test_data)
    eval_result = {'predictions': x, 'seed': seed}
    results.append(eval_result)
    with open(f'./results/{model_name}_{train_data_name}_seed_var_results.pkl', 'wb') as f:
        pickle.dump(results, f)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/scratch/1051127/ipykernel_196401/2779880192.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  seed_trainer = Trainer(


Step,Training Loss
1000,0.176400
2000,0.052300
3000,0.031200



Map:   0%|          | 0/61371 [00:00<?, ? examples/s]


Map:   1%|          | 695/61371 [00:00<00:08, 6875.55 examples/s]

...


Map:  98%|█████████▊| 60000/61371 [00:15<00:02, 514.30 examples/s] 


Map:  99%|█████████▉| 60798/61371 [00:15<00:00, 731.05 examples/s]


Map: 100%|██████████| 61371/61371 [00:15<00:00, 892.91 examples/s]


Map: 100%|██████████| 61371/61371 [00:15<00:00, 3947.61 examples/s]

              precision    recall  f1-score   support

       B-LOC       0.84      0.89      0.86     17495
      B-MISC       0.63      0.73      0.67     10657
       B-ORG       0.65      0.68      0.66     13041
       B-PER       0.90      0.92      0.91     15547
       I-LOC       0.70      0.66      0.68      5367
      I-MISC       0.66      0.35      0.46      7305
       I-ORG       0.92      0.71      0.80     18313
       I-PER       0.95      0.87      0.91     11086
           O       0.99      0.99      0.99   1011383

    accuracy                           0.97   1110194
   macro avg       0.80      0.76      0.77   1110194
weighted avg       0.97      0.97      0.97   1110194



In [31]:
with open(f'./results/{model_name}_{train_data_name}_seed_var_results.pkl', 'rb') as f:
    seed_res = pickle.load(f)

In [32]:
kappa_scores = []

for i, j in combinations(range(len(seed_res)), 2):
    preds_i = seed_res[i]['predictions']
    preds_j = seed_res[j]['predictions']

    kappa = cohen_kappa_score(preds_i, preds_j)
    kappa_scores.append(kappa)

    print(f"Cohen's kappa between seed {i} and seed {j}: {kappa:.6f}")

average_kappa = sum(kappa_scores) / len(kappa_scores)
print(f"\nAverage Cohen's kappa across all pairs: {average_kappa:.6f}")

Cohen's kappa between seed 0 and seed 1: 0.968839


Cohen's kappa between seed 0 and seed 2: 0.967721


Cohen's kappa between seed 0 and seed 3: 0.969990


Cohen's kappa between seed 0 and seed 4: 0.968262


Cohen's kappa between seed 1 and seed 2: 0.968510


Cohen's kappa between seed 1 and seed 3: 0.968212


Cohen's kappa between seed 1 and seed 4: 0.969408


Cohen's kappa between seed 2 and seed 3: 0.967933


Cohen's kappa between seed 2 and seed 4: 0.966671


Cohen's kappa between seed 3 and seed 4: 0.967895

Average Cohen's kappa across all pairs: 0.968344


In [33]:
data = np.array([seed['predictions'] for seed in seed_res])

alpha = krippendorff.alpha(reliability_data=data,
                           level_of_measurement='nominal')

print(f"\nKrippendorff’s alpha (nominal): {alpha:.4f}")


Krippendorff’s alpha (nominal): 0.9683
